<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">


# Project 4 
## Web Scraping for Indeed.com and Predicting Salaries

## Stage 1: Web scraping

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

### Web scraping - single page

In [2]:
# scrape first page of results
url = "http://www.indeed.co.uk/jobs?q=data+scientist+%C2%A310,000&l=london&start=0"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [3]:
# extract the job listings for first page of results
results_test = soup.find_all('div', class_='result')
print(results_test[0].prettify())

<div class="jobsearch-SerpJobCard unifiedRow row result" data-ci="362377375" data-empn="7967352176238747" data-jk="8d9fe9ee8cf09994" id="pj_8d9fe9ee8cf09994">
 <style>
  .jobcard_logo{margin:6px 0}.jobcard_logo img{width:auto;max-width:80px;max-height:30px}.jasxrefreshcombotst .jobcard_logo img{max-height:2rem;max-width:100%}
 </style>
 <h2 class="title">
  <a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0AZxsRq9b9GKxv_h7q1ABkCCcEffo3LKFWp04beLk-Eij8HBfBM2WRebKF84ZFn4HYAFsgZC-WoHArUNkDTjif58NCXnEGUrUC60UNspgr7HBqZQQoy94MgIlntClA3a8o-fERQTGoZsbRJsRPUM7_9FvmJzzEMuHOki92oGBWreCTpU3IHMi2Z4HQFue4DC8XgAknGzPOAiDTAmXnEiWb0TVbt__vxgCBE8cgncYmfRGMrpsHW3m8cSapyTkU2B_V5i8BT3RLdA2DBbPYxpAZryhpuaydh9lZIposGC1o83v-o4-a1l3dfNDw66zGQv8nGuaJU4rXhsC3uYp2oWRvgFSuummx44CpVgBw1F6nJuGJLeS7Ldu_QjLXk6Mdhwrd0E1cufyx-JBUYshHIJLdOgHEKDXtljVgI5Eto2q7BhAsaLoTXUd0_w9jiDdSu4-6kto5xXYHA-6gmwU_vFzvQ&amp;p=0&amp;fvj=1&amp;vjs=3" id="sja0" onclick="setRefineByCookie(['

In [4]:
# define functions to extract useful items
def extract_company_from_result(result):
    try:
        return result.find('span', class_='company').text.strip()
    except:
        return np.nan

def extract_location_from_result(result):
# sometimes location is contained in a 'div' not a 'span', check for both
    try:
        return result.find('span', class_='location').text.strip()
    except:
        try:
            return result.find('div', class_='location').text.strip()
        except:
            return np.nan

def extract_job_from_result(result):
    try:
        return result.find('a', class_='jobtitle').text.strip()
    except:
        return np.nan

def extract_salary_from_result(result):
    try:
        return result.find('span', class_='salaryText').text.strip()
    except:
        return np.nan
    
def extract_summary_from_result(result):
    try:
        return result.find('div', class_='summary').text.strip()
    except:
        return np.nan
    
def extract_rating_from_result(result):
    try:
        return result.find('span', class_='ratingsContent').text.strip()
    except:
        return np.nan
    
def extract_link_from_result(result):
    try:
        return result.find('a', class_='jobtitle')['href']
    except:
        return np.nan

In [11]:
# check results from scraping one page of results
companies = []
locations = []
jobs = []
salaries = []
summaries = []
ratings = []
links = []

for result in results_test:
    companies.append(extract_company_from_result(result))
    locations.append(extract_location_from_result(result))
    jobs.append(extract_job_from_result(result))
    salaries.append(extract_salary_from_result(result))
    summaries.append(extract_summary_from_result(result))
    ratings.append(extract_rating_from_result(result))
    links.append(extract_link_from_result(result))
    
df = pd.DataFrame({'company': companies,
                   'location': locations,
                   'job_title': jobs,
                   'salary': salaries,
                   'job_summary': summaries,
                   'company_rating': ratings,
                   'link': links})
df

,company,location,job_title,salary,job_summary,company_rating,link
0,NHS Midlands and Lancashire Commissioning Supp...,London SE1 6LH,Data Scientist,"£24,907 - £30,615 a year",Good knowledge of R. This is essential to mani...,4.5,/rc/clk?jk=195753e0562ed694&fccid=37c85e352ebb...
1,Lendable,London E1 6JE,Data Scientist,NaN,"Research new data sources, advise on costs / b...",NaN,/rc/clk?jk=86094c8dfc7c8a89&fccid=2c65da7ba6c7...
2,Novafutur,London,Junior Data Scientist,NaN,Being a data scientist would require the abili...,NaN,/rc/clk?jk=65cc8df228f482e3&fccid=d9cae5807f63...
3,Ernst & Young,London,Technology Consulting - Data Scientist,NaN,"Used a data visualisation technology, like Tab...",4.0,/rc/clk?jk=bfbb8063181c9c27&fccid=1544766d4c29...
4,Deliveroo,London,Data Scientist,NaN,Data scientists at Deliveroo report into our d...,3.4,/rc/clk?jk=ab2897f8e9b9c4f0&fccid=1d79a4290fb7...
5,Medialab,London,Junior Data Scientist Placement,NaN,Develop automated data processes using Python/...,NaN,/rc/clk?jk=aa102ebbe46abcbb&fccid=4f2893bccebb...
6,Asos.com,London,Data Scientist - Data Science & AI,NaN,Experience accessing and combining data from m...,3.0,/rc/clk?jk=7f6304de6c96752a&fccid=d4f2a7e16823...
7,Visa,London,Data Scientist,NaN,Mid-level experience solving problems with rea...,3.9,/rc/clk?jk=1bace1192b673784&fccid=a3f737e511d9...
8,Deutsche Bank,London,Anti-Fraud Bribery and Corruption Data Scientist,NaN,"Demonstrated experience in platforms (SAS, R e...",3.8,/rc/clk?jk=5025a182b2e067ac&fccid=f1d8e147024a...
9,UK Government - Government Statistical Service,London,Higher Statistical Data Scientist - across the...,"£28,996 a year",Knowledge of data processing to support analys...,NaN,/rc/clk?jk=8a3bbee163053507&fccid=dae1bd9f8c3a...


### Web scraping - multiple pages
Now that we know how to extract the relevant data from one page of search results, extend the scraping to collect more data. Do this in chunks and save the data into a CSV file at each stage. The table below shows the different searches performed. NB there will be many duplicates in these numbers.

|Job Title Searched|City / Cities Searched|Number of Results|
|:--|:--|--:|
|Data Scientist|London|785|
|Data Scientist|Manchester, Birmingham, Leeds, Reading, Liverpool, Glasgow|554|
|Data Scientist|Edinburgh, Sheffield, Newcastle, Bristol, Southampton, Nottingham|270|
|Data Scientist|Brighton, Cambridge, Oxford, Guildford, Cardiff, remote|528|
|Data Analyst|London|1,115|
|Data Analyst|Manchester, Birmingham, Leeds, Reading, Liverpool, Glasgow|914|
|Data Analyst|Edinburgh, Sheffield, Newcastle, Bristol, Southampton, Nottingham|602|
|Data Analyst|Brighton, Cambridge, Oxford, Guildford, Cardiff, remote|1,346|
|Data Engineer|London|255|
|Data Engineer|Manchester, Birmingham, Leeds, Reading, Liverpool, Glasgow|70|
|Data Engineer|Edinburgh, Sheffield, Newcastle, Bristol, Southampton, Nottingham|65|
|Data Engineer|Brighton, Cambridge, Oxford, Guildford, Cardiff, remote|231|
|Business Intelligence|London|925|
|Business Intelligence|London (2nd search with a later starting point) |1,127|
|Business Intelligence|London (3rd search with a later starting point) |2,000|
|Business Intelligence|Manchester, Birmingham, Leeds, Reading, Liverpool, Glasgow|2,983|
|Business Intelligence|Edinburgh, Sheffield, Newcastle, Bristol, Southampton, Nottingham|961|
|Business Intelligence|Brighton, Cambridge, Oxford, Guildford, Cardiff, remote|1,119|

In [5]:
cities = ['London', 'Manchester', 'Birmingham', 'Leeds', 'Reading', 'Liverpool', 'Glasgow', 
          'Edinburgh', 'Sheffield', 'Newcastle', 'Bristol', 'Southampton', 'Nottingham',
          'Brighton', 'Cambridge', 'Oxford', 'Guildford', 'Cardiff', 'remote']
job_titles = ['data+scientist', 'data+analyst', 'data+engineer', 'business+intelligence']

In [19]:
# scrape different cities and job titles to get more results
# there are 15 results per page, so increment in steps of 15
max_results_per_city = 50
job_title = 'data+scientist'
results = []

for city in set(['Brighton', 'Cambridge', 'Oxford', 'Guildford', 'Cardiff', 'remote']):
    for start in range(0, max_results_per_city, 15):
        url = f'http://www.indeed.co.uk/jobs?q={job_title}+%C2%A310,000&l={city}&fromage=3&start={start}'
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        results.append(soup.find_all('div', class_='result'))
        time.sleep(random.randint(2,5))

In [20]:
# add results to a dataframe
companies = []
locations = []
jobs = []
salaries = []
summaries = []
ratings = []
links = []

for page in results:
    for result in page:
        companies.append(extract_company_from_result(result))
        locations.append(extract_location_from_result(result))
        jobs.append(extract_job_from_result(result))
        salaries.append(extract_salary_from_result(result))
        summaries.append(extract_summary_from_result(result))
        ratings.append(extract_rating_from_result(result))
        links.append(extract_link_from_result(result))
    
df = pd.DataFrame({'company': companies,
                   'location': locations,
                   'job_title': jobs,
                   'salary': salaries,
                   'job_summary': summaries,
                   'company_rating': ratings,
                   'link': links})
df.shape

(128, 7)

In [21]:
# write df to csv file
with open('jobs.csv', 'a') as f:
    df.to_csv(f, mode='a', header=f.tell()==0)